In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('dataset/03262023_CNN_df.csv')

In [3]:
df.head()

,link,title,article,date
0,https://www.cnn.com/2023/03/26/us/delaware-riv...,Philadelphia officials say drinking water rema...,us,2023-03-26
1,https://www.cnn.com/2023/03/25/us/west-reading...,Death toll from explosion at Pennsylvania cand...,us,2023-03-26
2,https://www.cnn.com/2023/03/25/us/jonathan-maj...,Actor Jonathan Majors is arrested on assault c...,us,2023-03-26
3,https://www.cnn.com/2023/03/26/us/georgia-weat...,2 tigers recaptured after escaping Georgia saf...,us,2023-03-26
4,https://www.cnn.com/2023/03/26/us/indiana-univ...,12 injured in floor collapse at off-campus apa...,us,2023-03-26


In [4]:
df.article.unique()

array(['us', 'crime justice', 'Enery Environment', 'Space Science'],
      dtype=object)

In [5]:
import requests
import json
from bs4 import BeautifulSoup

In [6]:
def get_context_link(url):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        script = soup.find("script", {"type": "application/ld+json"})
        script_text = script.text
        data = json.loads(script_text)
        articleBody_value = data["articleBody"]
    except:
        articleBody_value = None
        
    if articleBody_value is None:
        try:
            response = requests.get(url)
            soup = BeautifulSoup(response.content, 'html.parser')
            script = soup.find("script", {"type": "application/ld+json"})
            script_text = script.text
            data = json.loads(script_text)
            articleBody_value = data["description"]
        except:
            articleBody_value = None
            
    if articleBody_value is None or '':
        try:
            sentences =[]
            response = requests.get(url)
            soup = BeautifulSoup(response.content, 'html.parser')
            script = soup.find("script", {"type": "application/ld+json"})
            script_text = script.text
            data = json.loads(script_text)
            for i in range(len(data['itemListElement'])):
                sentences.append(data['itemListElement'][i]['item']['caption'])
            articleBody_value = " ".join(sentences)
        except:
            articleBody_value = None    
    if articleBody_value is None:
        try:
            sentences =[]
            response = requests.get(url)
            soup = BeautifulSoup(response.content, 'html.parser')
            for p in soup.find('div', {'class':'article-body', 'itemprop':'articleBody'}).find_all('p'):
                if p.text.strip():
                    sentences.append(p.text.strip())
            articleBody_value = " ".join(sentences)
        except:
            articleBody_value = None
    if articleBody_value is None:
        try:
            sentences =[]
            response = requests.get(url)
            soup = BeautifulSoup(response.content, 'html.parser')
            for p in soup.find('div', {'class':'article-body | article w-100'}).find_all('p'):
                if p.text.strip():
                    sentences.append(p.text.strip())
            articleBody_value = " ".join(sentences)
        except:
            articleBody_value = None      
    if articleBody_value is None:
        try:
            sentences =[]
            response = requests.get(url)
            soup = BeautifulSoup(response.content, 'html.parser')
            for p in soup.find('article').find_all('p'):
                if p.text.strip():
                    sentences.append(p.text.strip())
            articleBody_value = " ".join(sentences)
        except:
            articleBody_value = None
    if articleBody_value is None:
        try:
            sentences =[]
            response = requests.get(url)
            soup = BeautifulSoup(response.content, 'html.parser')
            script = soup.find("div", {"class": "error-container"})
            if script.text.startswith('Uh-oh'):
                articleBody_value = 'ERROR on Pages'
            else:
                articleBody_value = 'Error'
        except:
            articleBody_value = None
    return articleBody_value

In [7]:
df_us = df[df.article=='us']
df_cj = df[df.article=='crime justice']
df_ee = df[df.article=='Enery Environment']
df_ss = df[df.article=='Space Science']

In [8]:
context_list_us =[]
for i in range(df_us.shape[0]):
    contxt = get_context_link(df_us.link[i+df_us.index[0]])
    context_list_us.append(contxt)

In [9]:
context_list_cj =[]
for i in range(df_cj.shape[0]):
    contxt = get_context_link(df_cj.link[i+df_cj.index[0]])
    context_list_cj.append(contxt)

In [10]:
context_list_ee =[]
for i in range(df_ee.shape[0]):
    contxt = get_context_link(df_ee.link[i+df_ee.index[0]])
    context_list_ee.append(contxt)

In [11]:
context_list_ss =[]
for i in range(df_ss.shape[0]):
    contxt = get_context_link(df_ss.link[i+df_ss.index[0]])
    context_list_ss.append(contxt)

In [12]:
import numpy as np

In [13]:
arr1 = np.array(context_list_us)
arr2 = np.array(context_list_cj)
arr3 = np.array(context_list_ee)
arr4 = np.array(context_list_ss)

In [14]:
combined_arr = np.concatenate((arr1, arr2, arr3, arr4))

In [15]:
df.shape, len(combined_arr)

((528, 4), 528)

In [16]:
df['content'] =combined_arr

In [17]:
df.head()

,link,title,article,date,content
0,https://www.cnn.com/2023/03/26/us/delaware-riv...,Philadelphia officials say drinking water rema...,us,2023-03-26,The City of Philadelphia says it is now confid...
1,https://www.cnn.com/2023/03/25/us/west-reading...,Death toll from explosion at Pennsylvania cand...,us,2023-03-26,A fourth person has been confirmed dead in an ...
2,https://www.cnn.com/2023/03/25/us/jonathan-maj...,Actor Jonathan Majors is arrested on assault c...,us,2023-03-26,"Actor Jonathan Majors, who has recently starre..."
3,https://www.cnn.com/2023/03/26/us/georgia-weat...,2 tigers recaptured after escaping Georgia saf...,us,2023-03-26,Two tigers have been recaptured after escaping...
4,https://www.cnn.com/2023/03/26/us/indiana-univ...,12 injured in floor collapse at off-campus apa...,us,2023-03-26,Twelve people were injured Saturday night when...


In [18]:
df.to_csv('dataset/03262023_CNN.csv', index=False)